In [1]:
!nvidia-smi

Fri Sep 20 12:28:26 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   42C    P8               9W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [4]:
%%writefile magnitude_tr.cu
 #include <thrust/device_vector.h>
#include <thrust/host_vector.h>
#include <thrust/transform_reduce.h>
#include <thrust/functional.h>
#include <cmath>
#include <iostream>
#include <chrono>

// Functor to square the values
struct square {
    __host__ __device__
    double operator()(const double& x) const {
        return x * x;
    }
};

int main() {
    const int n = 10000000;
    double value = 0.0;

    // Host vector
    thrust::host_vector<double> google_host(n);
    for (int i = 0; i < n; i++) {
        std::cin >> value;
        google_host[i] = value;
    }

    // Transfer to device
    thrust::device_vector<double> google_device = google_host;

    auto start = std::chrono::high_resolution_clock::now();

    // Use transform_reduce to calculate the sum of squares
    double sum_of_squares = thrust::transform_reduce(google_device.begin(), google_device.end(), square(), 0.0, thrust::plus<double>());

    // Compute magnitude as the square root of the sum of squares
    double magnitude = std::sqrt(sum_of_squares);

    auto end = std::chrono::high_resolution_clock::now();
    std::chrono::duration<double> elapsed_time = end - start;

    std::cout << "Magnitude (using transform_reduce): " << magnitude << std::endl;
    std::cout << "Elapsed time: " << elapsed_time.count() << " seconds" << std::endl;

    return 0;
}


Overwriting magnitude_tr.cu


In [6]:
!nvcc -arch=sm_70 -std=c++14 magnitude_tr.cu -o magnitude_tr

In [11]:
!./magnitude_tr < random_vector.txt

Magnitude (using transform_reduce): 1825.44
Elapsed time: 0.00103498 seconds


In [8]:
%%writefile magnitude_t_then_r.cu
#include <thrust/device_vector.h>
#include <thrust/host_vector.h>
#include <thrust/transform.h>
#include <thrust/reduce.h>
#include <cmath>
#include <iostream>
#include <chrono>

// Functor to square the values
struct square {
    __host__ __device__
    double operator()(const double& x) const {
        return x * x;
    }
};

int main() {
    const int n = 10000000;
    double value = 0.0;

    // Host vector
    thrust::host_vector<double> google_host(n);
    for (int i = 0; i < n; i++) {
        std::cin >> value;
        google_host[i] = value;
    }

    // Transfer to device
    thrust::device_vector<double> google_device = google_host;
    thrust::device_vector<double> squared_elements(n);

    auto start = std::chrono::high_resolution_clock::now();

    // Step 1: Square each element
    thrust::transform(google_device.begin(), google_device.end(), squared_elements.begin(), square());

    // Step 2: Sum the squared elements
    double sum_of_squares = thrust::reduce(squared_elements.begin(), squared_elements.end(), 0.0, thrust::plus<double>());

    // Step 3: Compute the magnitude
    double magnitude = std::sqrt(sum_of_squares);

    auto end = std::chrono::high_resolution_clock::now();
    std::chrono::duration<double> elapsed_time = end - start;

    std::cout << "Magnitude (using separate operations): " << magnitude << std::endl;
    std::cout << "Elapsed time: " << elapsed_time.count() << " seconds" << std::endl;

    return 0;
}

Writing magnitude_t_then_r.cu


In [9]:
!nvcc -arch=sm_70 -std=c++14 magnitude_t_then_r.cu -o magnitude_t_then_r

In [12]:
!./magnitude_t_then_r < random_vector.txt

Magnitude (using separate operations): 1825.44
Elapsed time: 0.00129333 seconds


In [13]:
%%writefile variance_tr.cu
#include <thrust/device_vector.h>
#include <thrust/host_vector.h>
#include <thrust/transform_reduce.h>
#include <thrust/functional.h>
#include <iostream>
#include <chrono>

// Functor to compute squared difference from mean
struct square_diff_from_mean {
    double mean;
    square_diff_from_mean(double _mean) : mean(_mean) {}

    __host__ __device__
    double operator()(const double& x) const {
        double diff = x - mean;
        return diff * diff;
    }
};

int main() {
    const int n = 10000000;
    double value = 0.0;

    thrust::host_vector<double> google_host(n);
    for (int i = 0; i < n; i++) {
        std::cin >> value;
        google_host[i] = value;
    }

    thrust::device_vector<double> google_device = google_host;

    auto start = std::chrono::high_resolution_clock::now();

    // Calculate the sum of elements
    double sum = thrust::reduce(google_device.begin(), google_device.end(), 0.0, thrust::plus<double>());

    // Compute the mean
    double mean = sum / n;

    // Use transform_reduce to calculate the sum of squared differences in a single step
    double sum_squared_diff = thrust::transform_reduce(google_device.begin(), google_device.end(),
                                                       square_diff_from_mean(mean), 0.0, thrust::plus<double>());

    // Calculate variance
    double variance = sum_squared_diff / n;

    auto end = std::chrono::high_resolution_clock::now();
    std::chrono::duration<double> elapsed_time = end - start;

    std::cout << "Variance (using transform_reduce): " << variance << std::endl;
    std::cout << "Elapsed time: " << elapsed_time.count() << " seconds" << std::endl;

    return 0;
}


Writing variance_tr.cu


In [14]:
!nvcc -arch=sm_70 -std=c++14 variance_tr.cu -o variance_tr

In [19]:
!./variance_tr < random_vector.txt

Variance (using transform_reduce): 0.0833208
Elapsed time: 0.00186854 seconds


In [15]:
%%writefile variance_t_then_r.cu
#include <thrust/device_vector.h>
#include <thrust/host_vector.h>
#include <thrust/transform.h>
#include <thrust/reduce.h>
#include <iostream>
#include <chrono>

// Functor to compute squared difference from mean
struct square_diff_from_mean {
    double mean;
    square_diff_from_mean(double _mean) : mean(_mean) {}

    __host__ __device__
    double operator()(const double& x) const {
        double diff = x - mean;
        return diff * diff;
    }
};

int main() {
    const int n = 10000000;
    double value = 0.0;

    thrust::host_vector<double> google_host(n);
    for (int i = 0; i < n; i++) {
        std::cin >> value;
        google_host[i] = value;
    }

    thrust::device_vector<double> google_device = google_host;
    thrust::device_vector<double> squared_diff(n);

    auto start = std::chrono::high_resolution_clock::now();

    // Step 1: Calculate the sum of elements
    double sum = thrust::reduce(google_device.begin(), google_device.end(), 0.0, thrust::plus<double>());

    // Step 2: Compute the mean
    double mean = sum / n;

    // Step 3: Transform the elements by subtracting the mean and squaring the difference
    thrust::transform(google_device.begin(), google_device.end(), squared_diff.begin(),
                      square_diff_from_mean(mean));

    // Step 4: Sum the squared differences
    double sum_squared_diff = thrust::reduce(squared_diff.begin(), squared_diff.end(), 0.0, thrust::plus<double>());

    // Step 5: Calculate the variance
    double variance = sum_squared_diff / n;

    auto end = std::chrono::high_resolution_clock::now();
    std::chrono::duration<double> elapsed_time = end - start;

    std::cout << "Variance (using separate operations): " << variance << std::endl;
    std::cout << "Elapsed time: " << elapsed_time.count() << " seconds" << std::endl;

    return 0;
}

Writing variance_t_then_r.cu


In [17]:
!nvcc -arch=sm_70 -std=c++14 variance_t_then_r.cu -o variance_t_then_r

In [20]:
!./variance_t_then_r < random_vector.txt

Variance (using separate operations): 0.0833208
Elapsed time: 0.00207674 seconds
